# Geo Visualisation
Please note that part of the code is taken from tutorials sheet written by the University of Melbourne lecturer Akirat Wang.

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import re

In [ ]:
gdf = gpd.read_file("../data/raw/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
income = pd.read_csv('../data/curated/income.csv')

In [ ]:
gdf = gdf[gdf['STE_NAME21'] == 'Victoria']

In [ ]:
sa2_name = gdf[['SA2_CODE21','SA2_NAME21']]

In [ ]:
gdf['geometry'] = gdf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf.head()

### SA2 map

In [ ]:
geoJSON = gdf[['SA2_CODE21', 'geometry']].drop_duplicates('SA2_CODE21').to_json()
print(geoJSON[:300])

In [ ]:
_map = folium.Map(location=[-37, 144], tiles="Stamen Terrain", zoom_start=7)


_map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))

_map.save('../../plots/SA2.html')
_map

### Average income for every SA2

In [ ]:
df = pd.read_csv("../data/raw/2021_GCP_SA2_for_VIC_short-header/2021 Census GCP Statistical Area 2 for VIC/2021Census_G33_VIC_SA2.csv")
df.head()

In [ ]:
cols = [col for col in df.columns if 'Tot' in col and 'Neg' not in col and 'stated' not in col]

In [ ]:
df1 = df[cols].drop(['Tot_Family_households','Tot_Non_family_households','Tot_Tot'],axis=1)
df1['SA2_CODE21'] = df['SA2_CODE_2021']
df1.head()

In [ ]:
df1.columns[0]

In [ ]:
avg = 0
df1['total_amount'] = 0
df1['family_amount'] = 0
for i in range(len(df1.columns)-3):
    if(i != 15):
        low = int(re.findall(r'[0-9]+', df1.columns[i])[0])
        upper = int(re.findall(r'[0-9]+', df1.columns[i])[1])
        avg = low + ((upper - low)/2)
    else:
        avg = 5500
    df1['total_amount'] = df1['total_amount'] + avg * df1.iloc[:, i]
    df1['family_amount'] = df1['family_amount'] + df1.iloc[:, i]
df1['avg'] = df1['total_amount']/df1['family_amount']

In [ ]:
df1.head()

In [ ]:
df2 = df1[['SA2_CODE21','avg']]
df2

In [ ]:
df2.to_csv('../data/curated/income.csv',index=False)

In [ ]:
gdf1 = gdf

In [ ]:
gdf1['SA2_CODE21'] = pd.to_numeric(gdf1['SA2_CODE21'])

In [ ]:
df = income.merge(gdf1[['SA2_CODE21', 'geometry']], on='SA2_CODE21')
df = df.drop('geometry', axis = 1)

In [ ]:
df['SA2_CODE21'] = df['SA2_CODE21'].astype(str)

In [ ]:
df

In [ ]:
m = folium.Map(location=[-37, 144], tiles="Stamen Terrain", zoom_start=7)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=df.reset_index(), # data source
    columns=['SA2_CODE21','avg'], # the columns required
    key_on='properties.SA2_CODE21', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Earnings AUD$'
)

c.add_to(m)

m.save('../../plots/income.html')
m